# board_calibration

- Camera calibration tool with aruco checker patterned board in input images

In [28]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3)
import glob
import os
import csv
import pickle
import json
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10.0, 10.0)
aruco = cv2.aruco

# Common paramters

# Create checker board
parameters = aruco.DetectorParameters_create()
dictionaryID = aruco.DICT_5X5_100
# dictionaryID = aruco.DICT_4X4_250
dictionary = aruco.getPredefinedDictionary(dictionaryID)
squareL = 0.028
markerL = 0.024
pixels_per_mm = 10 # for checker board image
A4size = (210, 297)
tb, lr = [5,5] # minimul margin (height, width) when printing in mm

# Calibration
calib_image_path = "../pictures/rsd435/"
calib_image_format = "jpg"
calib_result_save_format = "pkl"
calib_result_savedir = ""

In [29]:
def get_file_paths(file_dir, file_ext):
    path = file_dir + '*.' + file_ext
    file_names = [os.path.basename(r) for r in glob.glob(path)]
    file_paths = [file_dir+fs for fs in file_names]
    print(file_names)
    print(file_paths)
    return file_paths, file_names


def imshow_inline(img_name="", img=None):
    if img is None:
        if not img_name:
            print("Give imshow_inline an image name or an image.")
            return -1
        else:
            img = cv2.imread(img_name)
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [30]:
def max_within_upper(num, upper):
    i = 1
    while True:
        if num*i > upper:
            return ([i-1, int(num*(i-1))])
        else:
            i += 1

squareNumX, boardSizeX = max_within_upper(squareL*1000, A4size[0]- lr*2 ) # in mm
squareNumY, boardSizeY = max_within_upper(squareL*1000, A4size[1]- tb*2 ) # in mm


def get_board_image():
    board = aruco.CharucoBoard_create(squareNumX, squareNumY, squareL, markerL, dictionary)
    
    # The third parameter is the (optional) margin in pixels, so none of the markers are touching the image border.
    # Finally, the size of the marker border, similarly to drawMarker() function. The default value is 1.
    boardImage = board.draw((boardSizeX*pixels_per_mm, boardSizeY*pixels_per_mm), None, 0, 1) # 10 pixels/mm
    return(board, boardImage)


def add_margin(pil_img, tb_pixels, lr_pixels):
    width, height = pil_img.size
    new_width = width + lr_pixels
    new_height = height + tb_pixels
    result = Image.new(pil_img.mode, (new_width,new_height), (255))
    result.paste(pil_img, (int(lr_pixels/2), int(tb_pixels/2)))
    return result


def output_board_configparams_to_csv(output_filename):
    with open(output_filename, 'w', newline='') as csvfile:
        config_writer = csv.writer(csvfile, lineterminator='\n')
        config_writer.writerow(['dictionary ID', dictionaryID])
        config_writer.writerow(['square length', squareL])
        config_writer.writerow(['marker length', markerL])
        config_writer.writerow(['number of squares (x and y)', squareNumX, squareNumY])
        config_writer.writerow(['minimul margin (tb and lr)', tb, lr])


def create_ChArUco_board_in_A4size(board_name, outcsv=True):
    board, boardImage, = get_board_image()
    
    # Add the margin to the image
    tb_pixels = (A4size[1] - boardSizeY) * pixels_per_mm
    lr_pixels = (A4size[0] - boardSizeX) * pixels_per_mm
    boardImage_margin = np.asarray( add_margin(Image.fromarray(boardImage), tb_pixels, lr_pixels) )
    
    dirpath_board = "../board/"
    if not os.path.isdir(dirpath_board):
        os.mkdir(dirpath_board)
    cv2.imwrite(dirpath_board+board_name, boardImage_margin)
    imshow_inline(img_name=dirpath_board+board_name)
    
    if outcsv:
        filename_without_ext = os.path.splitext(board_name)[0]
        output_board_configparams_to_csv(dirpath_board+filename_without_ext+'.csv')

In [31]:
def detect_ChArUco_board(image_paths, outimg=True):
    for image_path in image_paths:
        checkerBoardImage = cv2.imread(image_path)

        # Detect ChArUco markers #
        markerCorners, markerIds  = [0,0]
        markerCorners, markerIds, rejectedImgPoints = aruco.detectMarkers(checkerBoardImage, dictionary)

        # Detect the checker board based on the detected marker, and draw the result #
        outputImage = checkerBoardImage.copy()
        if markerIds is None:
            break
        if markerIds.size > 0:
            charucoCorners, charucoIds = [0,0]
            cv2.aruco.drawDetectedMarkers(outputImage, markerCorners, markerIds)
            # charucoCorners, charucoIds = aruco.interpolateCornersCharuco(markerCorners, markerIds, checkerBoardImage, board)
            # outputImage = aruco.drawDetectedCornersCharuco(outputImage, charucoCorners, charucoIds)

        dirpath, file = os.path.split(image_path)
        dirpath_results = dirpath+"/aruco_detection_results/"
        if not os.path.isdir(dirpath_results):
            os.mkdir(dirpath_results)
        imshow_inline(img=outputImage)
        if outimg:
            cv2.imwrite(dirpath_results+file, outputImage)

In [32]:
def get_calibration_images(calib_img_paths, resimgs=False):
    calibImages = []
    for calib_img_path in calib_img_paths:
        calibImage = cv2.imread(calib_img_path)
        if calibImage is None:
            break
        if resimgs:
            calibImage = cv2.resize(calibImage,(1280,720))
        calibImages.append(calibImage)
    return calibImages

In [41]:
def show_calibration_result(calibrate_params):
    print("####################")
    retval, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsInstrinsics, stdDeviationsExtrinsics, perViewErrors = calibrate_params
    print("Final re-projection error : \n", retval)
    print("Camera matrix : \n", cameraMatrix)
    print("Vector of distortion coefficients : \n", distCoeffs)
    print("Vector of rotation vectors (see Rodrigues) : \n", rvecs)
    print("Vector of translation vectors : \n", tvecs)
    print("Vector of standard deviations estimated for intrinsic parameters : \n", stdDeviationsInstrinsics)
    print("Vector of standard deviations estimated for extrinsic parameters : \n", stdDeviationsExtrinsics)
    print("Vector of average re-projection errors : \n", perViewErrors)


def calibrate_with_ChArUco_board(calibImages, param_file_ex='.pkl'):
    board, boardImg = get_board_image()

    # Detect checker board intersection of ChArUco #
    allCharucoCorners = []
    allCharucoIds = []
    charucoCorners, charucoIds = [0,0]
    for calImg in calibImages:
        # Find ArUco markers #
        res = aruco.detectMarkers(calImg, dictionary)
        # Find ChArUco corners #
        if len(res[0])>0:
            res2 = cv2.aruco.interpolateCornersCharuco(res[0], res[1], calImg, board)
            if res2[1] is not None and res2[2] is not None and len(res2[1])>3:
                allCharucoCorners.append(res2[1])
                allCharucoIds.append(res2[2])

            cv2.aruco.drawDetectedMarkers(calImg,res[0],res[1])
        img = cv2.resize(calImg, None, fx=0.5, fy=0.5)
        # cv2.imshow('calibration image',img)
        # cv2.waitKey(0)
        
    cv2.destroyAllWindows()
    
    # Calibration and output errors #
    try:
        imgSize = calibImages[0].shape[:2]
        calibrate_params = cv2.aruco.calibrateCameraCharucoExtended(allCharucoCorners,allCharucoIds,board,imgSize,None,None)
    except:
        print("can not calibrate ...")

    # show_calibration_result(cal)
    retval, cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsInstrinsics, stdDeviationsExtrinsics, perViewErrors = calibrate_params
    tmp = [cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsInstrinsics, stdDeviationsExtrinsics]

    # Save the camera parameters #
    class MyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            else:
                return super(MyEncoder, self).default(obj)
    
    fname = "camera_param"
    calib_result_savedir = "../result"
    fpath = os.path.join(os.getcwd(), calib_result_savedir, fname)
    if calib_result_save_format == 'json':
        with open(fpath+'.json', mode='w') as f:
            data = {"camera_matrix": cameraMatrix.tolist(), "dist_coeff": distCoeffs.tolist(), "rvecs": rvecs, "tvecs": tvecs}
            json.dump(data,f,sort_keys=True,indent=4,cls=MyEncoder)
    else:
        with open(fpath+'.pkl', mode='wb') as f:
            pickle.dump(tmp, f, protocol=-1)
    
    print("Saved.")

In [42]:
def undistort(cam_param_path, images):
    # Read camera parameters as a pkl file #
    with open(cam_param_path, 'rb') as f:
        camera_params = pickle.load(f)

    cameraMatrix, distCoeffs, rvecs, tvecs, stdDeviationsInstrinsics, stdDeviationsExtrinsics = camera_params

    # Writing the camera matrix #
    imgSize = images[0].shape[:2]
    h,  w = imgSize
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(cameraMatrix,distCoeffs,(w,h),1,(w,h))
    print(newcameramtx, roi)

    for i, before_undistortImg in enumerate(images):
        # Undistort #
        dst = cv2.undistort(before_undistortImg, cameraMatrix, distCoeffs, None, newcameramtx)
        
        # Crop the image #
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
        dirpath = "./pictures/undistort_result/"
        if not os.path.isdir(dirpath):
            os.mkdir(dirpath)
        cv2.imwrite(dirpath+"undistorted"+ str(i+1) +'.png', dst)

In [43]:
def board_creation():
    board_name = "sample_board_"
    create_ChArUco_board_in_A4size(board_name+'.png')

In [44]:
def board_detection():
    detect_test_dir = "./tmp/" 
    pic_paths, pic_names = get_file_paths(detect_test_dir, 'jpg')
    detect_ChArUco_board(pic_paths)

In [45]:
def board_calibration():
    calib_image_paths, calib_image_names = get_file_paths(calib_image_path, calib_image_format)
    calibrate_with_ChArUco_board(get_calibration_images(calib_image_paths, resimgs=True))

In [46]:
def image_undistortion():
    cam_param_path = "./camera_param.pkl"
    undistort(cam_param_path, get_calibration_images(calib_img_paths))

In [47]:
if __name__ == '__main__':
    # board_creation()
    # board_detection()
    board_calibration()
    # image_undistortion()

['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
['../pictures/rsd435/1.jpg', '../pictures/rsd435/10.jpg', '../pictures/rsd435/11.jpg', '../pictures/rsd435/12.jpg', '../pictures/rsd435/13.jpg', '../pictures/rsd435/14.jpg', '../pictures/rsd435/15.jpg', '../pictures/rsd435/16.jpg', '../pictures/rsd435/17.jpg', '../pictures/rsd435/18.jpg', '../pictures/rsd435/19.jpg', '../pictures/rsd435/2.jpg', '../pictures/rsd435/20.jpg', '../pictures/rsd435/21.jpg', '../pictures/rsd435/22.jpg', '../pictures/rsd435/23.jpg', '../pictures/rsd435/24.jpg', '../pictures/rsd435/25.jpg', '../pictures/rsd435/26.jpg', '../pictures/rsd435/27.jpg', '../pictures/rsd435/28.jpg', '../pictures/rsd435/29.jpg', '../pictures/rsd435/3.jpg', '../pictures/rsd435/30.jpg', '../pictures/r